# Mount your google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install the specific version of 'httplib2'
there is an issue if we don't specify the version, this has been solved [here](https://stackoverflow.com/questions/60533230/pydrive-redirectmissinglocation-redirected-but-the-response-is-missing-a-locat)

In [ ]:
# we need to restart the session in the first time
!pip install httplib2==0.15.0

# Install the 'pydrive' library

In [ ]:
!pip install -U -q PyDrive
# simplifies the process of interacting with Google Drive API


# Authenticate and Authorize PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Function to copy files from source folder to destination folder
copy all the files in the source folder (not including the source folder itself) into the destination folder

In [ ]:
def copy_folder(source_folder_id, destination_folder_id):
    file_list = drive.ListFile({'q': f"'{source_folder_id}' in parents and trashed=false"}).GetList()
    for file in file_list:
        try:
            if file['mimeType'] == 'application/vnd.google-apps.folder':
                # Create a new folder in the destination folder
                new_folder = drive.CreateFile({
                    'title': file['title'],
                    'mimeType': 'application/vnd.google-apps.folder',
                    'parents': [{'id': destination_folder_id}]
                })
                new_folder.Upload()
                print(f'Folder "{file["title"]}" created.')
                # Recursively copy the contents of the folder
                copy_folder(file['id'], new_folder['id'])
            else:
                # Copy file to the destination folder
                new_file = drive.CreateFile({
                    'title': file['title'],
                    'parents': [{'id': destination_folder_id}]
                })
                file.GetContentFile('temp_file')
                new_file.SetContentFile('temp_file')
                new_file.Upload()
                print(f'File "{file["title"]}" copied.')
        except Exception as e:
            print(f'Error copying file "{file["title"]}": {e}')

# Function to move specific folder into trash can if exist

In [ ]:
# Function to check if a folder exists and move it to the trash if found
def trash_folder_if_exists(folder_name):
    # Search for the folder in the root directory
    query = f"title = '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and 'root' in parents and trashed = false"
    folder_list = drive.ListFile({'q': query}).GetList()

    if folder_list:
        # Folder found, move it to the trash
        for folder in folder_list:
            print(f'Folder "{folder["title"]}" found with ID: {folder["id"]}. Moving to trash...')
            folder.Trash()
        print(f'Folder "{folder_name}" moved to trash.')
    else:
        print(f'Folder "{folder_name}" does not exist.')

# modify 'source_folder_id'

In [ ]:
# Retrieve the source folder metadata
source_folder_id = '11iW9lRNIN7G4Unwsz48RIzTxAUmnGyvW'  # Replace with the actual source folder ID
source_folder = drive.CreateFile({'id': source_folder_id})
source_folder.FetchMetadata(fields='title')
source_folder_title = source_folder['title']

# check if the source_folder_title has already exists in your Google Drive
move the folder to the trash can if it exists

In [ ]:
trash_folder_if_exists(source_folder_title)

# begin the copy
this automatically create a new folder in your gdrive home, with the same name as the source_folder, and then copy all the folders and the files from the source_folder into the created folder in your gdrive home

requirements:
1. get a shareable link
2. The folder ID is a unique identifier in the shareable link. For example, in the link https://drive.google.com/drive/folders/1ABCDEFGH12345, the folder ID is 1ABCDEFGH12345.

In [ ]:
# Create a new folder in your Google Drive with the same name as the source folder and get its ID
new_folder = drive.CreateFile({'title': source_folder_title, 'mimeType': 'application/vnd.google-apps.folder'})
new_folder.Upload()
print(f'Folder "{new_folder["title"]}" created.')
destination_folder_id = new_folder['id']

# Copy the contents of the source folder to the new folder
copy_folder(source_folder_id, destination_folder_id)